# Finding high-frequency words tweets
In this notebook, a map-reduce procedure is used here to find words which have a high frequency in the unique tweets (this means retweets are excluded from analysis).

Download a tweets from  json from archive.org (https://archive.org/search.php?query=tweets). For every given minute there is .bz2 file and all files for every hour is given in separate folders. These files contains tweets in form of json.

Mapper and reducer files are below. The analysis is for one hour of tweets.
The first line in the cell save the rest of the cell as .py file. If you want to do this manually, remove the first line and save the cell content.

In [163]:
%%writefile mapper_vocab_freq.py 
#!/usr/bin/env python
import sys
import json

      
for currentTweet in sys.stdin:
    try:
        if currentTweet.strip() != "":
            currentTweet = currentTweet.lower().strip()  
            json_dic = json.loads(currentTweet)
            if 'text' in json_dic:
                currentTweetText = json_dic['text']
                # The following line replace all special character in the string with a space
                currentTweetText = currentTweetText.translate ({ord(ch): " " for ch in '"\'!@#$%^&*()[]{};:,./<>?\|`~-=_+0123456789'})
                list_of_words = currentTweetText.lower().split()
                for word in list_of_words:
                    if not (word in 'abcdefghijklmnopqrstuvwxyz'):
                        print(word,1)
    except:
        print('Error',1)

Overwriting mapper_vocab_freq.py


In [164]:
%%writefile reducer.py 
#!/usr/bin/env python
# source for reducer file: https://www.michael-noll.com/tutorials/writing-an-hadoop-mapreduce-program-in-python/
import sys

current_word = None
current_count = 0
word = None

for line in sys.stdin:
    line = line.strip()
    word, count = line.split(' ', 1)
    try:
        count = int(count)
    except ValueError:
        print('count_error')
    if current_word == word:
        current_count += count
    else:
        if current_word:
            text_to_print = current_word + ',' + str(current_count)
            print(text_to_print)
            #print(current_word, current_count)
        current_count = count
        current_word = word

if current_word == word:
    text_to_print = current_word + ',' + str(current_count)
    print(text_to_print)

Overwriting reducer.py


### run map reduce 
 run the following command in the command line:
 
 *bzcat *.bz2 | ./mapper_unique_tweets.py | sort -k1,1 | ./reducer.py*
 
 Another alternative is to write the above command in a shell script as follows. This .sh file write the result in output.csv

In [183]:
%%writefile mapreduce.sh
echo 'word,freq' > output.csv
bzcat *.bz2 | ./mapper_vocab_freq.py | sort -k1,1 | ./reducer.py >> output.csv

Overwriting mapreduce.sh


After that you can run the .sh file to create .csv file.

In [191]:
!mapreduce.sh

Now we can process the csv file

In [192]:
import csv
d = {}

with open('output.csv', newline='\n') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        d[row['word']] = int(row['freq'])

sorted_dict = dict(sorted(d.items(), key=lambda item: item[1],reverse=True))
number_of_distinct_words = len(sorted_dict)
number_of_distinct_words

277571

In [193]:
top_100 = {k: sorted_dict[k] for k in list(sorted_dict)[:100]}

In [194]:
top_100

{'Error': 169088,
 'rt': 116724,
 'https': 36194,
 'co': 35898,
 'the': 24548,
 'to': 16216,
 'and': 12572,
 'for': 12514,
 'in': 11430,
 'of': 10959,
 'is': 10946,
 'you': 10906,
 'this': 7687,
 'it': 6303,
 'me': 6118,
 'on': 6067,
 'my': 5370,
 'that': 5335,
 'que': 4878,
 'at': 4685,
 'la': 4492,
 'are': 4437,
 'we': 4406,
 'be': 4223,
 'with': 4082,
 'so': 3936,
 'all': 3796,
 'your': 3405,
 'bts': 3231,
 'not': 3197,
 'have': 3140,
 'en': 2996,
 'just': 2968,
 'do': 2866,
 'was': 2824,
 'but': 2814,
 'if': 2758,
 'like': 2700,
 'he': 2699,
 'as': 2636,
 '…': 2600,
 'can': 2573,
 'they': 2561,
 'what': 2469,
 'will': 2460,
 'from': 2412,
 'by': 2378,
 'one': 2363,
 'sscgd': 2226,
 'el': 2221,
 'amp': 2193,
 'who': 2127,
 'na': 2111,
 'halloween': 2093,
 'love': 2087,
 'up': 2059,
 'rajghat': 2006,
 'vote': 1987,
 'blackdiwali': 1965,
 'when': 1930,
 'happy': 1891,
 'how': 1886,
 'now': 1880,
 'or': 1850,
 'un': 1832,
 'about': 1831,
 'di': 1777,
 'se': 1769,
 'has': 1763,
 'voting

In [180]:
list(sorted_dict)

['Error',
 'rt',
 'https',
 'co',
 'the',
 'to',
 'and',
 'for',
 'in',
 'of',
 'is',
 'you',
 'this',
 'it',
 'me',
 'on',
 'my',
 'that',
 'que',
 'at',
 'la',
 'are',
 'we',
 'be',
 'with',
 'so',
 'all',
 'your',
 'bts',
 'not',
 'have',
 'en',
 'just',
 'do',
 'was',
 'but',
 'if',
 'like',
 'he',
 'as',
 '…',
 'can',
 'they',
 'what',
 'will',
 'from',
 'by',
 'one',
 'sscgd',
 'el',
 'amp',
 'who',
 'na',
 'halloween',
 'love',
 'up',
 'rajghat',
 'vote',
 'blackdiwali',
 'when',
 'happy',
 'how',
 'now',
 'or',
 'un',
 'about',
 'di',
 'se',
 'has',
 'voting',
 'people',
 'please',
 'time',
 'our',
 'twt',
 'day',
 'i’m',
 'more',
 'good',
 'an',
 'his',
 'out',
 'get',
 'ya',
 'new',
 'le',
 'es',
 'sa',
 'aku',
 'us',
 'am',
 'amas',
 'trump',
 'go',
 'know',
 'si',
 'los',
 'november',
 'it’s',
 'only',
 'favorite',
 'why',
 'there',
 'today',
 'lo',
 'her',
 'ko',
 'see',
 'their',
 'por',
 'official',
 'been',
 'want',
 'she',
 'year',
 'here',
 'don',
 're',
 'them',
 'eu